In [1]:
import os
import json
import numpy as np
import pandas as pd

In [8]:
os.chdir('data/CUB_200_2011/')
os.getcwd()

'/Users/minjshin/Desktop/eecs498_mlre/MLRE-REPLICATION-ICML-2023-Route-interpret-repeat/data/CUB_200_2011'

In [21]:
with open("attributes_names.txt", "r") as f:
    attribute_names = []
    lines = f.readlines()
    for line in lines:
        attr_name = line.split(" ")[1]
        attribute_names.append(attr_name)

In [22]:
attribute_names

['has_bill_shape::curved_(up_or_down)\n',
 'has_bill_shape::dagger\n',
 'has_bill_shape::hooked\n',
 'has_bill_shape::needle\n',
 'has_bill_shape::hooked_seabird\n',
 'has_bill_shape::spatulate\n',
 'has_bill_shape::all-purpose\n',
 'has_bill_shape::cone\n',
 'has_bill_shape::specialized\n',
 'has_wing_color::blue\n',
 'has_wing_color::brown\n',
 'has_wing_color::iridescent\n',
 'has_wing_color::purple\n',
 'has_wing_color::rufous\n',
 'has_wing_color::grey\n',
 'has_wing_color::yellow\n',
 'has_wing_color::olive\n',
 'has_wing_color::green\n',
 'has_wing_color::pink\n',
 'has_wing_color::orange\n',
 'has_wing_color::black\n',
 'has_wing_color::white\n',
 'has_wing_color::red\n',
 'has_wing_color::buff\n',
 'has_upperparts_color::blue\n',
 'has_upperparts_color::brown\n',
 'has_upperparts_color::iridescent\n',
 'has_upperparts_color::purple\n',
 'has_upperparts_color::rufous\n',
 'has_upperparts_color::grey\n',
 'has_upperparts_color::yellow\n',
 'has_upperparts_color::olive\n',
 'has_

In [11]:
# 200 lines (one class, same order as classes.txt), 312 space-separated cols (% of attribute in class, same order as attributes.txt)
attribute_per_class = pd.read_csv(os.path.join("attributes", "class_attribute_labels_continuous.txt"), sep=" ",
                                      header=None).to_numpy()

In [17]:
# 11788 total images (rows), each row is image_id - class_id
classes = pd.read_csv("image_class_labels.txt", sep=" ", header=None).to_numpy()[:, 1]

In [18]:
classes

array([  1,   1,   1, ..., 200, 200, 200], shape=(11788,))

In [41]:
attributes = np.zeros(shape=(11788, 312))
with open(os.path.join("attributes", "image_attribute_labels.txt"), "r") as f:
    lines = f.readlines()
    for line in lines:
        fields = line.split(" ")
        if fields[2] == "1":
            img_idx = int(fields[0]) - 1
            attr_idx = int(fields[1]) - 1
            attributes[img_idx][attr_idx] = 1

# total images (11788) * total attributes (312), each cell is 1 if present, 0 if not based on human label
np.save("original_attributes.npy", attributes)

In [27]:
very_denoised_attributes = np.zeros(shape=(11788, 312))
attribute_sparsity = np.zeros(attributes.shape[1])  # zeros of length 312

In [42]:
for c in range(1, 201):
    imgs = classes == c
    class_attributes = attribute_per_class[c - 1, :] > 50
    very_denoised_attributes[imgs, :] = class_attributes
    attribute_sparsity += class_attributes
attributes_to_filter = attribute_sparsity < 10
very_denoised_attributes = very_denoised_attributes[:, ~attributes_to_filter]

# total images (11788) * total attributes (312), each cell is 1 if over 50% of that class has the attr, 0 if less; based on does the class have that attribute (class_attribute_labels)
# Only the attributes that are present in >=10 classes are chosen
np.save("attributes.npy", very_denoised_attributes)

In [69]:
attributes_to_filter

array([ True, False,  True,  True, False,  True, False, False,  True,
        True, False,  True,  True,  True, False, False,  True,  True,
        True,  True, False, False,  True, False,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False,  True, False,  True, False,  True,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False,  True, False, False,  True, False,  True,  True,  True,
       False, False,  True,  True,  True,  True, False, False,  True,
       False,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,

In [44]:
# Filter attributes based on above criteria
with open("attributes_names.txt", "w") as f:
    json.dump(np.asarray(attribute_names)[~attributes_to_filter].tolist(), f)

In [46]:
# Clean up
for item in os.listdir():
    if item not in ["images", "images.txt", "README", "attributes_names.txt", "original_attributes.npy",  "attributes.npy"]:
        os.system(f"rm -r {item}")

In [ ]:
# Rename images
os.system("mv images/* .")
with open("images.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        fields = line.split(" ")
        img_idx = f"{int(fields[0]):05d}"
        img_name = fields[1][:-1]
        img_name = os.path.join(os.path.dirname(img_name), os.path.basename(img_name))
        new_name = os.path.join(os.path.dirname(img_name), img_idx) + os.path.splitext(img_name)[1]
        os.rename(img_name, new_name)
os.system("rm -r images")
os.remove("images.txt")
os.system("mv ../CUB_200_2011 ../data")
print("Images sorted and renamed")
print("Dataset configured correctly")

os.chdir(origin_dir)

In [50]:
attr = np.load("attributes.npy")

In [51]:
attr[0, :] == attr[1, :]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [52]:
og_attr = np.load("original_attributes.npy")

In [53]:
og_attr[0, :] == og_attr[1, :]

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,

In [55]:
attributes_to_filter

array([ True, False,  True,  True, False,  True, False, False,  True,
        True, False,  True,  True,  True, False, False,  True,  True,
        True,  True, False, False,  True, False,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False,  True, False,  True, False,  True,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False,  True, False, False,  True, False,  True,  True,  True,
       False, False,  True,  True,  True,  True, False, False,  True,
       False,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,

In [56]:
og_attr_filtered = og_attr[:, ~attributes_to_filter]

In [57]:
og_attr_filtered.shape

(11788, 108)

In [60]:
attr[0, :] != og_attr_filtered[0, :]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False])

In [64]:
total_wrong = list()
for i in range(attr.shape[0]):
    total_wrong.append(sum(attr[i, :] != og_attr_filtered[i, :]))
np.average(total_wrong)

np.float64(18.741431964709875)

In [67]:
filter_str = attributes_to_filter.tobytes()